In [29]:
from pathlib import Path
import json
import sys

import gtfstk as gt
import pandas as pd
import numpy as np
import shapely.geometry as sg
import folium

sys.path.append('../')

import gtfs_map_matcher as mm

%load_ext autoreload
%autoreload 2

DATA_DIR = Path('../data')
OUT_DIR = Path('../output')
SECRETS_PATH = Path('../secrets.json')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
points_by_key = {
    'bingo': [
        [174.843234, -41.137425],
        [174.828152, -41.130639],
    ],
    'bongo': [
        [174.80496399999998, -41.22333],
        [174.796785, -41.247057],
    ]
}

mm.map_match_google(points_by_key, mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'))

{'warningMessage': "Input path is too sparse. You should provide a path where consecutive points are closer to each other. Refer to the 'path' parameter in Google Roads API documentation.", 'snappedPoints': [{'location': {'longitude': 174.80501397147776, 'latitude': -41.22335545870424}, 'originalIndex': 0, 'placeId': 'ChIJ8dqN_LetOG0RzRAx6EPYfqQ'}, {'location': {'longitude': 174.7967170172291, 'latitude': -41.246993559684476}, 'originalIndex': 1, 'placeId': 'ChIJTcHE8XSuOG0RQLWtBmHvABM'}]}
{'warningMessage': "Input path is too sparse. You should provide a path where consecutive points are closer to each other. Refer to the 'path' parameter in Google Roads API documentation.", 'snappedPoints': [{'location': {'longitude': 174.8431671586028, 'latitude': -41.137424437287834}, 'originalIndex': 0, 'placeId': 'ChIJUzPxV25TP20RgBgWB2HvABM'}, {'location': {'longitude': 174.82814702694546, 'latitude': -41.13063985048523}, 'originalIndex': 1, 'placeId': 'ChIJoYfK2A5TP20RobqrBmHvABM'}]}


{'bingo': [[174.8431671586028, -41.137424437287834],
  [174.82814702694546, -41.13063985048523]],
 'bongo': [[174.80501397147776, -41.22335545870424],
  [174.7967170172291, -41.246993559684476]]}

In [2]:
path = DATA_DIR/'wellington_gtfs_20171016.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.assess_quality()

,indicator,value
0,num_route_short_names_duplicated,0
1,frac_route_short_names_duplicated,0
2,num_stop_time_dists_missing,0
3,frac_stop_time_dists_missing,0
4,num_direction_ids_missing,0
5,frac_direction_ids_missing,0
6,num_trips_missing_shapes,0
7,frac_trips_missing_shapes,0
8,num_departure_times_missing,0
9,frac_departure_times_missing,0


In [16]:
# Create sample points

t = feed.trips.merge(feed.routes)
cond = t['route_short_name'].str.len() == 2
cond &= t['route_type'] == 3
t = t[cond].copy()
%time spoints_1 = mm.sample_trip_points(feed, t.trip_id, 5)
%time spoints_2 = mm.sample_trip_points(feed, t.trip_id, 100)
%time spoints_3 = mm.sample_trip_points(feed, t.trip_id, point_dist=0.1)

print('num stop patterns = ', len(spoints_2))
pattern = list(spoints_1.keys())[0]


CPU times: user 1.1 s, sys: 0 ns, total: 1.1 s
Wall time: 1.1 s
CPU times: user 1.94 s, sys: 28 ms, total: 1.97 s
Wall time: 1.95 s
CPU times: user 2.55 s, sys: 0 ns, total: 2.55 s
Wall time: 2.53 s
num stop patterns =  181


In [ ]:
l1 = sg.LineString(spoints_1[pattern])
l2 = sg.LineString(spoints_2[pattern])
l3 = sg.LineString(spoints_3[pattern])

# Get actual trip shape
p = mm.get_stop_patterns(feed)
shid = p.loc[p['stop_pattern'] == pattern, 'shape_id'].unique()[0]
la = feed.build_geometry_by_shape(shape_ids=[shid], use_utm=False)[shid]

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(l1), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
folium.GeoJson(sg.mapping(l2), 
  style_function=lambda x: {'color': 'orange'}).add_to(mappy)
folium.GeoJson(sg.mapping(l3), 
  style_function=lambda x: {'color': 'yellow'}).add_to(mappy)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
mappy

In [ ]:
# Map match with Mapzen

mpoints = mm.map_match_mapzen({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'MAPZEN_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with OSRM

mpoints = mm.map_match_osrm({pattern: spoints_2[pattern]})[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with Mapbox, which uses OSRM

mpoints = mm.map_match_mapbox({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'MAPBOX_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Map match with Google

mpoints = mm.map_match_google({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [ ]:
# Test batch map match

n = 10
print('num requests =', n)
#test = {k: v for k, v in list(spoints_2.items())[:n]}
#api_key = mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY')
api_key = mm.get_secret(SECRETS_PATH, 'MAPBOX_API_KEY')

%time mpoints = mm.map_match(test, 'mapbox', api_key)

In [14]:
print(mm.get_num_map_matching_calls(feed))

426


In [ ]:
t = feed.trips.merge(feed.routes)
trip_id, shape_id = t.loc[t['route_type'] == 3, ['trip_id', 'shape_id']].iloc[0].values
print(trip_id, shape_id)
trip_ids = [trip_id]
%time feed2 = mm.create_shapes(feed, 'google', mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'), trip_ids=trip_ids)

In [ ]:
print(feed.shapes[feed.shapes.shape_id == shape_id])
print(feed2.shapes[feed2.shapes.shape_id == shape_id])